In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/avacado-price-prediction/Avocado.csv')

In [ ]:
df.head()

In [ ]:
df = df.drop('Unnamed: 0', axis = 1)

In [ ]:
df.Date.count()

In [ ]:
df.AveragePrice.describe()

In [ ]:
df.AveragePrice.hist()

In [ ]:
df['Total Volume'].describe()

In [ ]:
df['Total Volume'].hist()

In [ ]:

df['4046'].describe()

In [ ]:
df['4046'].hist()

In [ ]:
df['XLarge Bags'].describe()

In [ ]:
df['XLarge Bags'].hist()

In [ ]:
df.type.value_counts()

In [ ]:
df.region.value_counts()

In [ ]:
df.year.value_counts()

In [ ]:
df.columns

In [ ]:
df = df.rename(columns = {'Total Volume' : 'totalVolume', '4046' : 'PLU4046', '4225' : 'PLU4225', '4770' : 'PLU4770', 'Total Bags' : 'totalBags', \
          'Small Bags' : 'smallBags', 'Large Bags' : 'largeBags', 'XLarge Bags' : 'xLargeBags', 'Date' : 'date', 'AveragePrice' : 'averagePrice'})

In [ ]:
df.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scalerMinMax = MinMaxScaler()

In [ ]:
df.columns[2:10]

In [ ]:
scalerMinMax.fit(df[df.columns[2:10]])

In [ ]:
scalerMinMax.min_

In [ ]:
scalerMinMax.scale_

In [ ]:
scalerMinMax.data_min_

In [ ]:
scalerMinMax.data_max_

In [ ]:
scalerMinMax.data_range_

In [ ]:
df[df.columns[2:10]] = scalerMinMax.transform(df[df.columns[2:10]])

In [ ]:
df.head()

In [ ]:
df['type'] = df.type.astype('category')

In [ ]:
df = df.drop('year', axis = 1)

In [ ]:
df['type'].cat.categories

In [ ]:
df['type'] = df.type.cat.codes

In [ ]:
df['region'] = df.region.astype('category')

In [ ]:
df.region.cat.categories

In [ ]:
df['region'] = df.region.cat.codes

In [ ]:
df['date'] = pd.to_datetime(df.date)

In [ ]:
df.dtypes

In [ ]:
df.date.min()

In [ ]:
df.date.max()

In [ ]:
df.date.count()

In [ ]:
df.date.nunique()

In [ ]:
df['dayOfWeek'] = df.date.dt.dayofweek

In [ ]:
df['month'] = df.date.dt.month 

In [ ]:
df['firstPartOfMonth'] = df.date.dt.day <= 10

In [ ]:
df['secondPartOfMonth'] = ((df.date.dt.day > 10) & (df.date.dt.day <= 20))

In [ ]:
df['thirdPartOfMonth'] = df.date.dt.day > 20

In [ ]:
df['day'] = df.date.dt.day

In [ ]:
df.head()

In [ ]:
df.firstPartOfMonth.astype('int')

In [ ]:
df['firstPartOfMonth'] = df.firstPartOfMonth.astype('int')

In [ ]:
df['secondPartOfMonth'] = df.secondPartOfMonth.astype('int')

In [ ]:
df['thirdPartOfMonth'] = df.thirdPartOfMonth.astype('int')

In [ ]:
df.head()

In [ ]:
df = df.drop('date', axis = 1)

In [ ]:
df.corr()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df.averagePrice.count()

In [ ]:
df.averagePrice.hist()

In [ ]:
df.averagePrice.describe()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('averagePrice', axis = 1), df.averagePrice, test_size = 5000, random_state = 42)

In [ ]:
y_train.describe()

In [ ]:
y_train.hist()

In [ ]:
y_test.describe()

In [ ]:
y_test.hist()

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
tree = DecisionTreeRegressor(random_state = 42)

In [ ]:
tree.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
y_train_pred = tree.predict(X_train)
y_test_pred = tree.predict(X_test)

In [ ]:
mean_squared_error(y_train, y_train_pred, squared = False)

In [ ]:
mean_squared_error(y_test, y_test_pred, squared = False)

In [ ]:
from sklearn.metrics import max_error

In [ ]:
print('train set: ', max_error(y_train, y_train_pred))
print('test set: ', max_error(y_test, y_test_pred))

In [ ]:
abs(y_test - y_test_pred).describe()

In [ ]:
abs(y_test - y_test_pred).hist()

In [ ]:
sum(abs(y_test - y_test_pred) > 0.5)

In [ ]:
y_test.count()

In [ ]:
from sklearn.metrics import explained_variance_score

In [ ]:
print('train set: ', explained_variance_score(y_train, y_train_pred))
print('test set: ', explained_variance_score(y_test, y_test_pred))

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
print('train set: ', r2_score(y_train, y_train_pred))
print('test set: ', r2_score(y_test, y_test_pred))

In [ ]:
def checkMetrics(model):
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    print('Train Set:')
    print('RMSE :', mean_squared_error(y_train, y_train_pred, squared = False))
    print('Max Error: ', max_error(y_train, y_train_pred))
    print('R2-score: ', r2_score(y_train, y_train_pred))
    print('Number of errors > 0.5: ', str(sum(abs(y_train - y_train_pred) > 0.5)), end = '\n\n\n')
    
    print('Test Set:')
    print('RMSE :', mean_squared_error(y_test, y_test_pred, squared = False))
    print('Max Error: ', max_error(y_test, y_test_pred))
    print('R2-score: ', r2_score(y_test, y_test_pred))
    print('Number of errors > 0.5: ', str(sum(abs(y_test - y_test_pred) > 0.5)))

In [ ]:
checkMetrics(tree)

In [ ]:
from sklearn.linear_model import ElasticNet

In [ ]:
elasNet = ElasticNet(random_state = 42)

In [ ]:
elasNet.fit(X_train, y_train)

In [ ]:
checkMetrics(elasNet)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
reg = LinearRegression()
reg.fit(X_train, y_train)
checkMetrics(reg)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
checkMetrics(tree)

In [ ]:
gBoost = GradientBoostingRegressor(random_state = 42)
gBoost.fit(X_train, y_train)
checkMetrics(gBoost)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
forest = RandomForestRegressor(random_state = 42)
forest.fit(X_train, y_train)
checkMetrics(forest)

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

In [ ]:
ada = AdaBoostRegressor(base_estimator = DecisionTreeRegressor(random_state = 42), random_state = 42)
ada.fit(X_train, y_train)
checkMetrics(ada)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
params = {
    'base_estimator__max_depth' : [None, 5, 8, 10, 12, 15, 20, 25],\
    'base_estimator__max_features' : [None, 'log2', 'auto', 'sqrt'],\
    'base_estimator__max_leaf_nodes' : [None, 2, 4, 6, 8, 10, 15],\
    'n_estimators' : [25, 50, 75, 100, 150, 250],\
    'loss' : ['linear', 'square', 'exponential']
}

In [ ]:
from sklearn.metrics import SCORERS

In [ ]:
SCORERS.keys()

In [ ]:
base_tree = DecisionTreeRegressor(random_state = 42) 
base_ada = AdaBoostRegressor(base_estimator = base_tree, random_state = 42) 
clf = RandomizedSearchCV(random_state = 42, param_distributions = params, estimator = base_ada, scoring = 'neg_root_mean_squared_error', n_iter = 200, return_train_score = True)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
clf.best_params_

In [ ]:
clf.best_score_

In [ ]:
best_model = clf.best_estimator_

In [ ]:
checkMetrics(best_model)

In [ ]:
ada = AdaBoostRegressor(base_estimator = DecisionTreeRegressor(random_state = 42, max_depth = 8), random_state = 42, n_estimators = 100, loss = 'square')
ada.fit(X_train, y_train)
checkMetrics(ada)

In [ ]:
params = {
    'max_depth' : [None, 5, 8, 10, 12, 15, 20, 25],\
    'max_features' : [None, 'log2', 'auto', 'sqrt'],\
    'max_leaf_nodes' : [None, 2, 4, 6, 8, 10, 15],\
    'n_estimators' : [25, 50, 75, 100, 150, 250]
}

In [ ]:
base_forest = RandomForestRegressor(random_state = 42)
clf = RandomizedSearchCV(random_state = 42, param_distributions = params, estimator = base_forest,\
                         scoring = 'neg_root_mean_squared_error', n_iter = 200, return_train_score = True)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
clf.best_params_

In [ ]:
best_forest = clf.best_estimator_

In [ ]:
checkMetrics(best_forest)

AdaBoost
Train Set:
RMSE : 0.15704756892047753
Max Error:  0.4219480519480525
R2-score:  0.8483424500450756
Number of errors > 0.5:  0


Test Set:
RMSE : 0.19002481465267532
Max Error:  0.9565432098765427
R2-score:  0.77541986395577
Number of errors > 0.5:  44

Ada
Test Set:
RMSE : 0.19002481465267532
Max Error:  0.9565432098765427
R2-score:  0.77541986395577
Number of errors > 0.5:  44
Forest
Test Set:
RMSE : 0.16037730275544326
Max Error:  0.9559999999999997
R2-score:  0.8400307473893369
Number of errors > 0.5:  60

In [ ]:
from sklearn.feature_selection import RFECV

In [ ]:
rfe = RFECV(estimator = best_forest, min_features_to_select = 5, verbose = 4, scoring = 'neg_root_mean_squared_error')

In [ ]:
rfe.fit(X_train, y_train)

In [ ]:
rfe.support_

In [ ]:
rfe.ranking_

In [ ]:
X_train.columns

In [ ]:
X_train.columns[rfe.support_]

In [ ]:
X_train_sub = rfe.transform(X_train)
X_test_sub = rfe.transform(X_test)

In [ ]:
forest = RandomForestRegressor(random_state = 42, n_estimators = 75, max_features = 'auto')

In [ ]:
forest.fit(X_train_sub, y_train)

In [ ]:
def checkSubMetrics(model):
    y_train_pred = model.predict(X_train_sub)
    y_test_pred = model.predict(X_test_sub)
    
    print('Train Set:')
    print('RMSE :', mean_squared_error(y_train, y_train_pred, squared = False))
    print('Max Error: ', max_error(y_train, y_train_pred))
    print('R2-score: ', r2_score(y_train, y_train_pred))
    print('Number of errors > 0.5: ', str(sum(abs(y_train - y_train_pred) > 0.5)), end = '\n\n\n')
    
    print('Test Set:')
    print('RMSE :', mean_squared_error(y_test, y_test_pred, squared = False))
    print('Max Error: ', max_error(y_test, y_test_pred))
    print('R2-score: ', r2_score(y_test, y_test_pred))
    print('Number of errors > 0.5: ', str(sum(abs(y_test - y_test_pred) > 0.5)))

In [ ]:
checkSubMetrics(forest)